## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-18-58-33 +0000,nyt,"After TikTok Deal, Chinese Firms Search for a ...",https://www.nytimes.com/2026/01/23/business/ti...
1,2026-01-23-18-57-37 +0000,nyt,Trump’s Push to Take Greenland Holds Promise a...,https://www.nytimes.com/2026/01/23/world/europ...
2,2026-01-23-18-57-18 +0000,bbc,Chris Mason: Starmer's strongest rebuke yet fo...,https://www.bbc.com/news/articles/c9dvnlvzjw7o...
3,2026-01-23-18-57-04 +0000,nyt,5 Arrested in Shooting of Indiana Judge Steven...,https://www.nytimes.com/2026/01/23/us/indiana-...
4,2026-01-23-18-54-58 +0000,bbc,Harry says sacrifices by Nato troops in Afghan...,https://www.bbc.com/news/articles/clym2l7d75eo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2405/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,81
5,new,21
122,ice,20
10,greenland,17
118,minnesota,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
240,2026-01-23-02-31-45 +0000,cbc,Trump says he's withdrawing invitation for Car...,https://www.cbc.ca/news/politics/trump-board-o...,134
238,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...,134
338,2026-01-22-21-19-43 +0000,bbc,"Board of Peace, Zelensky and Musk - What happe...",https://www.bbc.com/news/videos/c78vvvn4qnlo?a...,132
40,2026-01-23-17-54-06 +0000,nypost,Ryan Wedding’s is 6th top ten most wanted fugi...,https://nypost.com/2026/01/23/us-news/ryan-wed...,132
222,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...,131


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
240,134,2026-01-23-02-31-45 +0000,cbc,Trump says he's withdrawing invitation for Car...,https://www.cbc.ca/news/politics/trump-board-o...
358,60,2026-01-22-19-43-16 +0000,nyt,Judge Rejects DOJ’s Criminal Complaint Against...,https://www.nytimes.com/2026/01/22/us/politics...
198,55,2026-01-23-08-08-54 +0000,bbc,Is the US-TikTok deal a new reality for China'...,https://www.bbc.com/news/articles/c6200mz054yo...
32,54,2026-01-23-18-10-10 +0000,nypost,DHS officials debunk viral claims ICE used 5-y...,https://nypost.com/2026/01/23/us-news/dhs-offi...
238,53,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
39,52,2026-01-23-17-57-06 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
109,46,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked, ...",https://www.washingtonpost.com/world/2026/01/2...
245,38,2026-01-23-02-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
233,35,2026-01-23-03-00-00 +0000,wsj,When asked what connects his two spheres of in...,https://www.wsj.com/business/media/dana-whites...
340,33,2026-01-22-21-15-36 +0000,nypost,Rep. Chip Roy tears into Jack Smith over Arcti...,https://nypost.com/2026/01/22/us-news/texas-re...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
